# Cartera Multifactorial Paramétrica

## Optimización de Portafolio usando Factores: Value, Momentum, Size y Beta

Este notebook ejecuta una optimización de cartera que maximiza la exposición a factores fundamentales, respetando una restricción de volatilidad.

In [ ]:
import sys
sys.path.insert(0, '../codigo')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Parametrico import OptimizacionMultifactorial
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('[OK] Librerias importadas')

## PASO 1: Configuración Inicial

Definimos los parámetros clave para la optimización multifactorial:

In [ ]:
config = {
    "file_path": "../../data/prod_long_sharpe_u60_20260125_v1_train_dataset.xlsx",
    "start_t": 750,
    "beta_min": 1.2,
    "annualization": 252,
    "ridge": 1e-6,
    "sigma_target": 0.175,
    "alpha_prefs": {
        "Value": 0.40,
        "Momentum": 0.30,
        "Size": -0.20,
        "Beta": 0.10
    }
}

alpha_prefs_desc = """
VECTOR DE PREFERENCIAS ALPHA:
  Value:    0.40  → Preferencia por activos BARATOS (1/PB alto)
  Momentum: 0.30  → Preferencia por GANADORES recientes
  Size:    -0.20  → Preferencia por SMALL CAPS (negativo en log-cap)
  Beta:     0.10  → Preferencia por AGRESIVOS dentro del universo
  
OTROS PARÁMETROS:
  Beta Min:       1.2  → Solo activos con beta agresiva (>= 1.2)
  Sigma Target:   17.5% → Volatilidad máxima de la cartera
  Start T:        750  → Período inicial para análisis (omite setup histórico)
  Annualization:  252  → Factor de conversión diario a anual
"""

print(alpha_prefs_desc)

## PASO 2: Instanciar y Ejecutar Optimización

In [ ]:
opt = OptimizacionMultifactorial(
    file_path=config["file_path"],
    start_t=config["start_t"],
    beta_min=config["beta_min"],
    annualization=config["annualization"],
    ridge=config["ridge"],
    sigma_target=config["sigma_target"],
    alpha_prefs=config["alpha_prefs"]
)

w_opt, resultados = opt.ejecutar_completo()

opt.printear_resultados()

## PASO 3: Detalle de Pesos y Top Posiciones

In [ ]:
df_pesos = opt.get_pesos_detallados()

print("\nTOP 15 POSICIONES (por peso):")
print("="*80)
print(df_pesos.head(15).to_string(index=False))

print(f"\n\nRESTUMEN DE PESOS:")
print(f"  - Activos con peso > 0.1%: {(df_pesos['Weight'] > 0.001).sum()}")
print(f"  - Suma total: {df_pesos['Weight'].sum():.6f}")
print(f"  - Máximo peso individual: {df_pesos['Weight'].max()*100:.2f}%")
print(f"  - Mínimo peso positivo: {df_pesos[df_pesos['Weight']>0]['Weight'].min()*100:.4f}%")

## PASO 4: Gráfico 1 - Distribución de Pesos (Top 20)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

top20 = df_pesos.head(20)
colors = plt.cm.viridis(np.linspace(0, 1, len(top20)))

ax.barh(range(len(top20)), top20['Weight_Pct'], color=colors)
ax.set_yticks(range(len(top20)))
ax.set_yticklabels([f"Ticker {int(t)}" for t in top20['Ticker']], fontsize=10)
ax.set_xlabel('Peso (%)', fontsize=12, fontweight='bold')
ax.set_title('Top 20 Posiciones - Cartera Multifactorial', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

for i, (idx, row) in enumerate(top20.iterrows()):
    ax.text(row['Weight_Pct'] + 0.1, i, f"{row['Weight_Pct']:.2f}%", 
            va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('parametrico_top20_pesos.png', dpi=300, bbox_inches='tight')
plt.show()

print('[OK] Gráfico 1 guardado: parametrico_top20_pesos.png')

## PASO 5: Gráfico 2 - Exposiciones Factoriales por Activo (Top 15)

In [ ]:
top15_tickers = df_pesos.head(15)['Ticker'].values
X_top15 = opt.X.loc[top15_tickers].copy()
X_top15['Ticker'] = [f"T{int(t)}" for t in top15_tickers]

fig, ax = plt.subplots(figsize=(14, 10))

X_plot = X_top15[['Value', 'Momentum', 'Size', 'Beta']].values
tickers_labels = X_top15['Ticker'].values

x = np.arange(len(tickers_labels))
width = 0.2

ax.bar(x - 1.5*width, X_plot[:, 0], width, label='Value', alpha=0.8)
ax.bar(x - 0.5*width, X_plot[:, 1], width, label='Momentum', alpha=0.8)
ax.bar(x + 0.5*width, X_plot[:, 2], width, label='Size', alpha=0.8)
ax.bar(x + 1.5*width, X_plot[:, 3], width, label='Beta', alpha=0.8)

ax.set_xlabel('Activos (Top 15 por Peso)', fontsize=12, fontweight='bold')
ax.set_ylabel('Exposición Factorial', fontsize=12, fontweight='bold')
ax.set_title('Exposiciones Factoriales: Value, Momentum, Size, Beta', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(tickers_labels, fontsize=9)
ax.legend(loc='upper right', fontsize=11)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('parametrico_exposiciones_factoriales.png', dpi=300, bbox_inches='tight')
plt.show()

print('[OK] Gráfico 2 guardado: parametrico_exposiciones_factoriales.png')

## PASO 6: Gráfico 3 - Matriz de Correlación (Top 15 activos)

In [ ]:
R_top15 = opt.R[top15_tickers]
corr_matrix = R_top15.corr()

fig, ax = plt.subplots(figsize=(12, 10))

ticker_labels = [f"T{int(t)}" for t in top15_tickers]
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            xticklabels=ticker_labels, yticklabels=ticker_labels,
            square=True, cbar_kws={'label': 'Correlación'}, ax=ax)

ax.set_title('Matriz de Correlación - Top 15 Activos (por Peso)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('parametrico_correlacion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print('[OK] Gráfico 3 guardado: parametrico_correlacion_matrix.png')

## PASO 7: Gráfico 4 - Distribución de Factores en Cartera Completa

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

factors = ['Value', 'Momentum', 'Size', 'Beta']
factor_exposures = []

for factor in factors:
    factor_exposures.append(opt.w_opt @ opt.X[factor].values)

colors_factors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']

axes[0, 0].hist(opt.X['Value'], bins=30, alpha=0.7, color='#e74c3c', edgecolor='black')
axes[0, 0].axvline(factor_exposures[0], color='darkred', linestyle='--', linewidth=2.5, label=f'Cartera: {factor_exposures[0]:.3f}')
axes[0, 0].set_title('Factor VALUE - Distribución Universal', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Exposición (z-score)')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

axes[0, 1].hist(opt.X['Momentum'], bins=30, alpha=0.7, color='#3498db', edgecolor='black')
axes[0, 1].axvline(factor_exposures[1], color='darkblue', linestyle='--', linewidth=2.5, label=f'Cartera: {factor_exposures[1]:.3f}')
axes[0, 1].set_title('Factor MOMENTUM - Distribución Universal', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Exposición (z-score)')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

axes[1, 0].hist(opt.X['Size'], bins=30, alpha=0.7, color='#2ecc71', edgecolor='black')
axes[1, 0].axvline(factor_exposures[2], color='darkgreen', linestyle='--', linewidth=2.5, label=f'Cartera: {factor_exposures[2]:.3f}')
axes[1, 0].set_title('Factor SIZE - Distribución Universal', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Exposición (z-score)')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

axes[1, 1].hist(opt.X['Beta'], bins=30, alpha=0.7, color='#f39c12', edgecolor='black')
axes[1, 1].axvline(factor_exposures[3], color='darkred', linestyle='--', linewidth=2.5, label=f'Cartera: {factor_exposures[3]:.3f}')
axes[1, 1].set_title('Factor BETA - Distribución Universal', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Exposición (Beta)')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('parametrico_distribucion_factores.png', dpi=300, bbox_inches='tight')
plt.show()

print('[OK] Gráfico 4 guardado: parametrico_distribucion_factores.png')

## PASO 8: Exportar Vector de Pesos (Completo - 60 activos)

In [ ]:
pesos_completos = np.zeros(60)
for ticker, weight in zip(opt.eligible_tickers, opt.w_opt):
    pesos_completos[int(ticker) - 1] = weight

df_export = pd.DataFrame({
    'Activo': [f'Activo_{i}' for i in range(1, 61)],
    'Ticker': list(range(1, 61)),
    'Peso': [round(p, 4) for p in pesos_completos],
    'Peso_Pct': [round(p*100, 2) for p in pesos_completos]
})

df_export.to_csv('../vector_pesos_parametrico.csv', index=False)

print('[OK] Pesos exportados: vector_pesos_parametrico.csv')
print()
print('VECTOR COMPLETO (60 ACTIVOS - primeros 30):')
print(df_export.head(30).to_string(index=False))

## PASO 9: Resumen Ejecutivo y Análisis de Sectores

In [ ]:
print("\n" + "="*80)
print("RESUMEN EJECUTIVO - CARTERA MULTIFACTORIAL")
print("="*80)

print(f"\n1. CONSTRUCCION DE CARTERA:")
print(f"   - Universo inicial: 60 activos")
print(f"   - Filtro Beta >= {config['beta_min']}: {len(opt.eligible_tickers)} activos elegibles")
print(f"   - Activos con peso > 0%: {(df_pesos['Weight'] > 0).sum()}")
print(f"   - Activos con peso > 1%: {(df_pesos['Weight'] > 0.01).sum()}")

print(f"\n2. METRICAS DE RIESGO-RETORNO:")
print(f"   - Volatilidad cartera: {resultados['vol_p']*100:.2f}% (Target: {config['sigma_target']*100:.2f}%)")
print(f"   - Beta cartera: {resultados['beta_p']:.3f} (Agresiva)")
print(f"   - Exposición Value: {resultados['exp_value']:+.3f}")
print(f"   - Exposición Momentum: {resultados['exp_mom']:+.3f}")
print(f"   - Exposición Size: {resultados['exp_size']:+.3f}")

print(f"\n3. CONCENTRACION:")
print(f"   - Top 1 posición: {df_pesos.iloc[0]['Weight_Pct']:.2f}%")
print(f"   - Top 5 posiciones: {df_pesos.head(5)['Weight'].sum()*100:.2f}%")
print(f"   - Top 10 posiciones: {df_pesos.head(10)['Weight'].sum()*100:.2f}%")

sector_exposure = df_pesos.groupby('Sector')['Weight'].sum().sort_values(ascending=False)
print(f"\n4. EXPOSICION POR SECTOR (Top 8):")
for sector, weight in sector_exposure.head(8).items():
    print(f"   - {sector}: {weight*100:.2f}%")

print("\n" + "="*80)

## PASO 10: Gráfico 5 - Concentración Acumulada (Curva de Lorenz)

In [ ]:
pesos_ordenados = np.sort(df_pesos['Weight'].values)[::-1]
pesos_acum = np.cumsum(pesos_ordenados)
posiciones = np.arange(1, len(pesos_acum) + 1)
pct_posiciones = posiciones / len(pesos_acum) * 100

fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(pct_posiciones, pesos_acum*100, linewidth=2.5, color='#2c3e50', marker='o', markersize=3, label='Cartera')
ax.plot([0, 100], [0, 100], 'r--', linewidth=2, label='Distribución Uniforme (ref.)')

ax.fill_between(pct_posiciones, pesos_acum*100, pct_posiciones, alpha=0.2, color='#2c3e50')

ax.set_xlabel('% de Activos (ordenados por peso)', fontsize=12, fontweight='bold')
ax.set_ylabel('% de Peso Acumulado', fontsize=12, fontweight='bold')
ax.set_title('Concentración de Cartera (Curva de Lorenz)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11, loc='upper left')

ax.axhline(y=50, color='gray', linestyle=':', alpha=0.5)
ax.axhline(y=80, color='gray', linestyle=':', alpha=0.5)
ax.axvline(x=20, color='gray', linestyle=':', alpha=0.5)

ax.text(5, 55, '50%', fontsize=10, color='gray')
ax.text(5, 85, '80%', fontsize=10, color='gray')

plt.tight_layout()
plt.savefig('parametrico_concentracion.png', dpi=300, bbox_inches='tight')
plt.show()

print('[OK] Gráfico 5 guardado: parametrico_concentracion.png')

## PASO 11: Gráfico 6 - Exposición por Sector

In [ ]:
sector_exposure = df_pesos.groupby('Sector')['Weight_Pct'].sum().sort_values(ascending=False).head(10)

fig, ax = plt.subplots(figsize=(12, 7))

colors = plt.cm.Set3(np.linspace(0, 1, len(sector_exposure)))
bars = ax.bar(range(len(sector_exposure)), sector_exposure.values, color=colors, edgecolor='black', linewidth=1.5)

ax.set_xticks(range(len(sector_exposure)))
ax.set_xticklabels(sector_exposure.index, rotation=45, ha='right', fontsize=11)
ax.set_ylabel('Peso en Cartera (%)', fontsize=12, fontweight='bold')
ax.set_title('Exposición por Sector - Top 10', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

for i, (bar, val) in enumerate(zip(bars, sector_exposure.values)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig('parametrico_exposicion_sectores.png', dpi=300, bbox_inches='tight')
plt.show()

print('[OK] Gráfico 6 guardado: parametrico_exposicion_sectores.png')